# Durable Rules Magic and Utilities

The [durable.rules](https://github.com/jruizgit/rules/) Python package provides a powerful rule based system that allows uses to create rule sets, assert facts and events, and reason on from those assertions according to the rules.

For use as a simple demonstrator in an educational context, the syntax and workflow are rather complex.

The intention behind the [`durable_rules_tools`](https://github.com/innovationOUtside/durable_rules_magic) package is to provide a range of utilities and simple IPython magic support for demonstrating the use of rule based systems in a Jupyter notebook in simple contexts, specifically ones based around "subject, predicate, object" triple style reasoning.

The approach taken may not scale for working with complex rulesets, or even in the general case. That said, the simplifications may provide a useful on-ramp for getting started using the package before moving on to using it directly.

## Defining Rulesets and Assertions

In the first instance, we need to define a set of rules against we can then then assert persistent facts and ephemeral events.

In [1]:
from durable.lang import ruleset, delete_state, when_all, assert_fact, retract_fact, post, c, m

The `durable.rules` README provides the following example of a ruleset and a series of assertions:

In [2]:
with ruleset('animal'):
    @when_all(c.first << (m.predicate == 'eats') & (m.object == 'flies'),
              (m.predicate == 'lives') & (m.object == 'water') & (m.subject == c.first.subject))
    def frog(c):
        c.assert_fact({ 'subject': c.first.subject, 'predicate': 'is', 'object': 'frog' })

    @when_all(c.first << (m.predicate == 'eats') & (m.object == 'flies'),
              (m.predicate == 'lives') & (m.object == 'land') & (m.subject == c.first.subject))
    def chameleon(c):
        c.assert_fact({ 'subject': c.first.subject, 'predicate': 'is', 'object': 'chameleon' })

    @when_all((m.predicate == 'eats') & (m.object == 'worms'))
    def bird(c):
        c.assert_fact({ 'subject': c.m.subject, 'predicate': 'is', 'object': 'bird' })

    @when_all((m.predicate == 'is') & (m.object == 'frog'))
    def green(c):
        c.assert_fact({ 'subject': c.m.subject, 'predicate': 'is', 'object': 'green' })

    @when_all((m.predicate == 'is') & (m.object == 'chameleon'))
    def grey(c):
        c.assert_fact({ 'subject': c.m.subject, 'predicate': 'is', 'object': 'grey' })

    @when_all((m.predicate == 'is') & (m.object == 'bird'))
    def black(c):
        c.assert_fact({ 'subject': c.m.subject, 'predicate': 'is', 'object': 'black' })

    @when_all(+m.subject)
    def output(c):
        print('Fact: {0} {1} {2}'.format(c.m.subject, c.m.predicate, c.m.object))


assert_fact('animal', { 'subject': 'Kermit', 'predicate': 'eats', 'object': 'flies' })
assert_fact('animal', { 'subject': 'Kermit', 'predicate': 'lives', 'object': 'water' })
assert_fact('animal', { 'subject': 'Greedy', 'predicate': 'eats', 'object': 'flies' })
assert_fact('animal', { 'subject': 'Greedy', 'predicate': 'lives', 'object': 'land' })
assert_fact('animal', { 'subject': 'Tweety', 'predicate': 'eats', 'object': 'worms' });  

Fact: Kermit eats flies
Fact: Kermit is green
Fact: Kermit is frog
Fact: Kermit lives water
Fact: Greedy eats flies
Fact: Greedy is grey
Fact: Greedy is chameleon
Fact: Greedy lives land
Fact: Tweety is black
Fact: Tweety is bird
Fact: Tweety eats worms


The rule syntax is visually complex, with decorators, logical tests and `dict` definitions including large amounts of boilerplate text.

If you duplicate an assertion, an error will be raised:

In [3]:
assert_fact('animal', { 'subject': 'Tweety', 'predicate': 'eats', 'object': 'worms' });  

MessageObservedException: {'subject': 'Tweety', 'predicate': 'eats', 'object': 'worms'}

Facts that have been asserted can be retracted, which means they can then be reasserted:

In [4]:
retract_fact('animal', { 'subject': 'Tweety', 'predicate': 'eats', 'object': 'worms' })
assert_fact('animal', { 'subject': 'Tweety', 'predicate': 'eats', 'object': 'worms' });
# If an error has been logged, a log object is returned; the ";" suppresses it's display.

Fact: Tweety eats worms


We can reset the state associated with a ruleset by deleting the current state over it:

In [5]:
delete_state('animal', None)

We now have a clean state to work with:

In [6]:
assert_fact('animal', { 'subject': 'Tweety', 'predicate': 'eats', 'object': 'worms' });  

Fact: Tweety is black
Fact: Tweety is bird
Fact: Tweety eats worms


Unlike facts, which are persistent once asserted and until retracted, events are ephemeral; once an event has been processed, its original assertion is retracted:

In [7]:
post('animal', { 'subject': 'Felix', 'predicate': 'eats', 'object': 'birds' })
post('animal', { 'subject': 'Felix', 'predicate': 'eats', 'object': 'birds' })

Fact: Felix eats birds
Fact: Felix eats birds


{'sid': '0', 'id': 'sid-0', '$s': 1}

(Facts that are asserted then retracted are possible in the form of event assertions.)

If you rerun the ruleset definition cell, a different sort of error will be raised, in the form of an `Exception: Ruleset with name animal already registered`. Within a Pyhton environment, only a single instance of a ruleset with a particular name is allowed, although many rulesets with unique names are supported.

### Simplifying Ruleset Definitions

Several helper functions are provided in `durable_rules_tools.rules_utils` for defining rulesets in a robust way (in educational settings, we often need to be as defensive as possible).

In [8]:
from durable_rules_tools.rules_utils import new_ruleset, SPO, Subject, Set

The `new_ruleset()` function creates a new, unique ruleset name that significantly minimiseds ruleset name. Making it easy to create new, unique names simplifies the use of such names.

To simplify tests against particular predicate and object settings, such as  `(m.predicate == 'eats') & (m.object == 'worms')`, we can instead test against `subject_('eats', 'worms')` (corresponding `object_(subj, pred)` and `predicate_(subj, obj)` tests are also defined).

We can also define simplifications on an *ad hoc* basis. For example, if we want to repeatedly test for a particular object value and that it described using an `is` predicate:

In [9]:
def _when_is(obj):
    """Test a specified object value is set against an 'is' predicate."""
    return (m.predicate == 'is') & (m.object == obj)

#That could be simplified further as: return subject_('is', obj)

When asserting facts, the function `SPO(subj, obj, pred)` allows us to replace statements of the form:

```python
{ 'subject': c.m.subject, 'predicate': 'is', 'object': 'bird' }
```

with statements of the form:

```python
SPO(c, 'is', 'bird)
```

where position is used to assume `c.m.subject`, `c.m.predicate` and `c.m.object` values where a string value is not passed.

As a further simplification, when asserting a fact in a rule body, eg as `c.assert_fact(SPO(c, 'can', 'fly'))`, we can rewrite it more simply as `Set(c, '? : can : fly' )`.

The previous ruleset now becomes:

In [10]:
RULESET = new_ruleset()
with ruleset(RULESET):
    @when_all(c.first << Subject('eats', 'flies'),
              Subject('lives', 'water') & (m.subject == c.first.subject))
    def frog(c):
        c.assert_fact(SPO(c.first.subject, 'is', 'frog'))

    @when_all(c.first << Subject('eats', 'flies'),
              Subject('lives', 'land') & (m.subject == c.first.subject))
    def chameleon(c):
        c.assert_fact(SPO(c.first.subject, 'is', 'chameleon'))

    @when_all(Subject('eats', 'worms'))
    def bird(c):
        Set(c, '? : is : bird')

    @when_all(Subject('is', 'frog'))
    def green(c):
        Set(c, '? : is : green')

    @when_all(Subject('is', 'chameleon'))
    def grey(c):
        Set(c, '? : is : grey')

    @when_all(Subject('is', 'bird'))
    def black(c):
        Set(c, '? :is : black')
        
    @when_all(Subject("is", "bird"))
    def can_fly(c):
        Set(c, '? : can : fly' )

    @when_all(+m.subject)
    def output(c):
        print('Fact: {0} {1} {2}'.format(c.m.subject, c.m.predicate, c.m.object))

In the original form, fact assertions are made in the form:

```python
assert_fact('animal', { 'subject': 'Kermit', 'predicate': 'eats', 'object': 'flies' })
```

We simplify these as simple text strings of the form `Kermit : eats : flies`, running them against a specified ruleset whose name is passed as an argument to a block cell magic that prefixes the assertion statement list.

The cell magic is called as `%%assert_facts -r RULESET`.

Enable the magic using:

In [11]:
%reload_ext durable_rules_tools

Then call the magic to test the assertions against the rulebase:

In [12]:
%%assert_facts -r RULESET
Kermit : eats : worms

Fact: Kermit can fly
Fact: Kermit is black
Fact: Kermit is bird
Fact: Kermit eats worms


Precede a fact with a `#` to ignore it.

If you rerun the above cell, no error is thrown. By default, state is cleared when the magic is invoked.

If you want to preseve state, you can pass the `--no-reset` flag. Note that this *will* result in an assertion error if you try to reassert a fact previously asserted:

In [13]:
%%assert_facts -r RULESET --no-reset
Kermit : eats : worms
#Kermit : eats : worms

/Users/tonyhirst/Documents/GitHub/durable_rules_magic/durable_rules_tools/rules_utils.py:74: UserWarning: Assertion error: is ['Kermit', 'eats', 'worms'] already asserted?
  warnings.warn(f"Assertion error: is {_statement} already asserted?")


Precede a fact with a `-` to retract rather than assert it:

In [16]:
%%assert_facts -r RULESET --no-reset
Tweety : eats : worms
- Tweety : eats : worms
Tweety : eats : worms

Fact: Tweety can fly
Fact: Tweety is black
Fact: Tweety is bird
Fact: Tweety eats worms
Fact: Tweety eats worms


## Events

We can post multiple events using the `%%post_events -r RULESET` magic:

In [14]:
%%post_events -r RULESET
Kermit : eats : flies
Kermit : eats : flies

Fact: Kermit eats flies
Fact: Kermit eats flies


## Facts and Events

We can specify asserted and retracted facts as well as events in the `%%facts_and_events` magic. Prefix each line with an appropriate action character:

- use `*` to assert a fact;
- use `-` to retract a fact;
- use `%` to post an event;

In [20]:
%%facts_and_events -r RULESET
Kermit : eats : worms
- Kermit : eats : worms
% Kermit : eats : worms
* Kermit : eats : worms

Fact: Kermit can fly
Fact: Kermit is black
Fact: Kermit is bird
Fact: Kermit eats worms


## Set accept mutliple args

how about letting `Set` be sensitive as to the args it takes?

```
Set(c, '? : can : fly' )
Set(c, (c.first.subject, 'can', 'fly') )
```